## 分析transformer模型的参数量、计算量、中间激活、KV cache

Layer Norm和Batch Norm都是常用的归一化方法，它们的公式和常见放置位置如下：

Batch Norm的公式为：
$$\hat{x}^{(k)}=\frac{x^{(k)}-\mathrm{E}[x^{(k)}]}{\sqrt{\mathrm{Var}[x^{(k)}]+\epsilon}}$$
$$y^{(k)}=\gamma^{(k)}\hat{x}^{(k)}+\beta^{(k)}$$

其中，$x^{(k)}$表示第$k$个特征维度的输入，$\mathrm{E}[x^{(k)}]$和$\mathrm{Var}[x^{(k)}]$分别表示在batch维度上第$k$个特征的均值和方差，$\gamma^{(k)}$和$\beta^{(k)}$是可学习的缩放和偏移参数。[1][4][5]

Batch Norm常放置在卷积层或全连接层之后，激活函数之前。在CNN中，Batch Norm是在每个特征维度上归一化，对batch中HxW个元素求均值和方差。[1][4][8]

Layer Norm的公式为：
$$\hat{x}^{(i)}=\frac{x^{(i)}-\mathrm{E}[x^{(i)}]}{\sqrt{\mathrm{Var}[x^{(i)}]+\epsilon}}$$
$$y^{(i)}=\gamma\hat{x}^{(i)}+\beta$$

其中，$x^{(i)}$表示第$i$个样本的所有特征，$\mathrm{E}[x^{(i)}]$和$\mathrm{Var}[x^{(i)}]$表示第$i$个样本特征的均值和方差，$\gamma$和$\beta$是可学习的缩放和偏移参数，对所有样本共享。[1][4][5]

Layer Norm常用于RNN、Transformer等处理序列数据的模型中，它在每个样本上对所有特征做归一化。在Transformer中，Layer Norm常放置在每个子层（Self-Attention、Feed Forward）之后。[1][3][5]

综上，Batch Norm和Layer Norm的主要区别在于：
1. Batch Norm在batch维度归一化，Layer Norm在特征维度归一化  
2. Batch Norm的缩放偏移参数$\gamma$和$\beta$与特征维度$k$相关，Layer Norm的$\gamma$和$\beta$对所有特征共享
3. Batch Norm常用于CNN，Layer Norm常用于RNN和Transformer

Layer Norm不依赖batch大小，适合变长序列的归一化，因此在NLP任务中应用更广泛。

Citations:
[1] https://www.cnblogs.com/shine-lee/p/11989612.html
[2] https://qkunai.com/2023/08/28/%E6%89%8B%E6%8A%8A%E6%89%8B%E5%B8%A6%E4%BD%A0%E4%BA%86%E8%A7%A3%E5%8F%B1%E8%AF%A7%E9%A3%8E%E4%BA%91%E7%9A%84-transformer%E6%9E%B6%E6%9E%84%E6%9E%B6%E6%9E%84%E5%88%86%E6%9E%90%E4%BB%A3%E7%A0%81/
[3] https://wqw547243068.github.io/chatgpt_mimic
[4] https://www.cnblogs.com/tian777/p/17911800.html
[5] https://www.jiqizhixin.com/articles/2019-07-09-5
[6] https://blog.csdn.net/qq_36560894/article/details/115017087
[7] https://blog.csdn.net/Flying_sfeng/article/details/116540763
[8] https://blog.csdn.net/m0_37192554/article/details/85049433
[9] https://www.cnblogs.com/LittleHann/p/17318509.html
[10] https://github.com/DA-southampton/NLP_ability/blob/master/%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86/Transformer/NLP%E4%BB%BB%E5%8A%A1%E4%B8%AD-layer-norm%E6%AF%94BatchNorm%E5%A5%BD%E5%9C%A8%E5%93%AA%E9%87%8C.md
[11] https://www.zhihu.com/question/487766088/answer/2422936310
[12] https://arxivtools.blob.core.windows.net/xueshuxiangzipaperhtml/2024_3_1/2402.19231.pdf
[13] https://wap.sciencenet.cn/blog-3396477-1407467.html?mobile=1
[14] https://blog.csdn.net/qq1145520074/article/details/79151948
[15] https://www.cvmart.net/community/detail/1569
[16] https://zh-v2.d2l.ai/d2l-zh-pytorch.pdf

## 简单单层参数量分析

1. 权重矩阵 $W^{Q}$、$W^{K}$、$W^{V}$  
shape: (h, h)  
参数量： $4*h^{2} + 4*h$  
2. MLP部分  
常规为4倍的前馈网络，两层的参数量均为 $4*h^{2}$，偏置为前层的$4h$ 以及后层的$h$，一共$8h^{2} + 5h$
3. Layer Norm
在Multihead Attention和FFN之后，各有一个，参数量为缩放因子和偏移因子，形状均为$h$，总大小 $4h$

每一个Transformer模块的参数量共： $12h^{2} + 13h$

## 单层计算量分析
输入映射，不考虑位编码和词表映射的计算量，输入shape (batch_size, seq_len, hidden_size) -> (b, s, h)  
考虑乘法和加法的数量级

1.1 Q、K、V映射 

(b, s, h) X (h, h) (b, s, h) -> (b, s, h) $2bsh^{2}*3$

1.2 Multihead attention score 的计算 

$Q\times K^{T}$ (b, s, h)x(b, h, s) -> (b, s, s) $2bs^{2}h$

1.3 计算 Value 

$\rm{Softmax(score)} \times V$ (b, s, s)x(b, s, h) -> (b, s, h) $2bs^{2}h$

1.4 线性映射(本质是让Transformer知道应该关注Multihead的哪些部分)
```
在Multi-head Attention的实现中，在计算完attention score与Value的乘积后，还需要经过一次线性映射，主要有以下几个原因：

1. 将多头attention的输出维度还原回与输入维度一致。由于在Multi-head Attention中，输入被拆分成多个头(head)分别计算attention，每个头的维度是原始维度除以头数。当把多个头的结果concat起来后，维度扩大到了原始维度的倍数。因此需要一个线性变换层将维度降回与输入一致，以便后续的残差连接和Layer Normalization操作。[1][4][5][8]

2. 让模型学习不同头的attention结果的重要性。通过线性变换层，可以自动学习到不同头输出的重要程度，相当于给每个头的结果都乘以一个权重，再求和。这样可以让模型根据任务自适应地调整每个头的权重。[4][5]

3. 提高模型的表达能力。这个线性变换实际上扮演了融合多头信息的角色，它可以捕捉不同子空间之间的相关性，从而得到一个信息更加丰富的向量表示。类似于CNN中使用1x1卷积来融合不同特征图的信息。[1][5]

4. 防止attention结果过度拟合到某一个头上。如果没有这一步线性变换，模型可能会过度依赖某个头的结果，从而影响泛化性能。加入线性变换层相当于在多个头之间做了一个平滑。[4]

综上，Multi-head Attention后的线性变换是非常必要和关键的一步，它让多头attention的结果维度与输入一致，并融合了不同头的信息，提高了模型的表达能力，让attention机制更加健壮。这也体现了Transformer设计的精巧之处。

Citations:
[1] https://www.cnblogs.com/xianbin7/p/11349312.html
[2] https://www.zhihu.com/question/592626839
[3] https://bbs.huaweicloud.com/blogs/382395
[4] https://jimmy-walker.gitbook.io/tensorflow/attention-and-transformer
[5] https://blog.csdn.net/qq_42363032/article/details/125988557
[6] https://developer.aliyun.com/article/1462204
[7] http://www.c-s-a.org.cn/csa/article/html/6944
[8] https://blog.csdn.net/None_Pan/article/details/106414724
```

(b, s, h)x(h, h) -> $2bsh^{2}$

1.5 FFN / MLP 网络结构

(b, s, h)x(h, 4h) 和 (b, s, 4h)x(4h, s) -> $16bsh^{2}$

故一个Transformer层的计算量大约是 $24bsh^{2} + 4bs^{2}h$

1.6 `Logits`输出映射

(b, s, h)x(h, V) -> $2bshV$




## 中间激活的显存占用（Bytes）
这里主要考虑的是前向传递中计算得到，并且在后向传递过程中所需要的张量。前向和后向计算中，视参数使用2bytes的浮点数进行保存。注意mask矩阵的每个元素仅为 1 byte。

1. 输入 x 

(b, s, h) -> $2bsh$ 

2. Multihead Attention 中的 Q / K / V

(b, s, h)*3 -> $6bsh$

3. 计算`Score`时需要保留 $Q\times K^{T}$的中间结果和最终在V上的attention

(b, head_num, s, h_per_head)x(b, head_num, h_per_head, s) -> (b, head_num, s, s) -> $2bs^{2} head num$*2

4. droup_out保留droupout mask
$bs^{2}head_num$

4. 输出映射保留其输入以及droupout 的mask

输入 (b, s, h) -> $2bsh$ / dropout (b, s, h) -> $bsh$

5. MLP的
(b, s, h) 第一个线性层输入/ (b, s, 4h) 激活函数输入 / (b, s, 4h)第二个线性层输入 / (b, s, h) / dropout mask 
$19bsh$

6. Layer Norm
4*(b, s, h) 


## 实现Llama Attention

In [23]:
import math
from typing import *

import torch
import torch.nn as nn


class LlamaAttention(nn.Module):
    def __init__(self, n_head: int,
                       hidden_dim: int,
                       bias: bool= False):
        super().__init__()
        assert hidden_dim%n_head == 0, "n_head must can be devided by hidden_dim."
        self.hidden_dim = hidden_dim
        self.head_dim = hidden_dim//n_head
        self.n_head = n_head
        
        self.q_proj = nn.Linear(hidden_dim, hidden_dim, bias=bias)
        self.k_proj = nn.Linear(hidden_dim, hidden_dim, bias=bias)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim, bias=bias)
        self.o_proj = nn.Linear(hidden_dim, hidden_dim, bias=bias)

    def forward(self,
                hidden_states: torch.Tensor,
                past_kv_states: Optional[Tuple[torch.Tensor]] = None,
                attention_mask: Optional[torch.Tensor] = None,
                use_cache: bool = False,
                output_attentions: bool = False
                ):
        bsz, seq_len, _ = hidden_states.shape
        
        q = self.q_proj(hidden_states).view(bsz, seq_len, self.n_head, self.head_dim).transpose(1, 2)
        k =  self.k_proj(hidden_states).view(bsz, seq_len, self.n_head, self.head_dim).transpose(1, 2)
        v = self.v_proj(hidden_states).view(bsz, seq_len, self.n_head, self.head_dim).transpose(1, 2)

        if past_kv_states is not None:
            k, v  = torch.cat([past_kv_states[0], k], dim=2), torch.cat([past_kv_states[1], v], dim=2)        
        past_kv_states = [k, v] if use_cache else None

        attentions = torch.matmul(q, k.transpose(2, 3))/math.sqrt(self.head_dim)

        if attention_mask is not None:
            attentions  = attentions + attention_mask
            attention_dtype_min = torch.Tensor(torch.finfo(attentions.dtype).min, device=attentions.device, dtype=attentions.dtype)
            attentions = torch.max(attentions, attention_dtype_min)
            print(attentions.shape)
        
        attentions_out = torch.matmul(nn.functional.softmax(attentions, dim=-1, dtype=torch.float32), v)
        attentions_out = attentions_out.transpose(1, 2).reshape(bsz, seq_len, self.hidden_dim)
        attentions_out = self.o_proj(attentions_out)
        
        if not output_attentions:
            attentions = None
        return attentions_out, attentions,  past_kv_states

1. 前馈中，先进行seq_len维度和n_head维度的transpose;
2. attention_weights 在进行softmax操作时需要使用torch.float32作为结果的type，最终维度为-1；
3. use_cache用于表征最后是否保留past_kv_states，而attention_output则用于表征最后是否保留attention的weights;
4. 如果需要做dropout，则应该在计算attention_weights之后；

## 实现Dot Product

In [ ]:
# Efficient implementation equivalent to the following:
def scaled_dot_product_attention(query, key, value, attn_mask=None, dropout_p=0.0, is_causal=False, scale=None) -> torch.Tensor:
    L, S = query.size(-2), key.size(-2)
    scale_factor = 1 / math.sqrt(query.size(-1)) if scale is None else scale
    attn_bias = torch.zeros(L, S, dtype=query.dtype)
    if is_causal:
        assert attn_mask is None
        temp_mask = torch.ones(L, S, dtype=torch.bool).tril(diagonal=0)
        attn_bias.masked_fill_(temp_mask.logical_not(), float("-inf"))
        attn_bias.to(query.dtype)

    if attn_mask is not None:
        if attn_mask.dtype == torch.bool:
            attn_bias.masked_fill_(attn_mask.logical_not(), float("-inf"))
        else:
            attn_bias += attn_mask
    attn_weight = query @ key.transpose(-2, -1) * scale_factor
    attn_weight += attn_bias
    attn_weight = torch.softmax(attn_weight, dim=-1)
    attn_weight = torch.dropout(attn_weight, dropout_p, train=True)
    return attn_weight @ value

In [22]:
attention_layer = LlamaAttention(4, 32, True)
x = torch.randn(2, 32, 32)

attention_layer(x)

(tensor([[[ 0.1652, -0.0233,  0.1016,  ...,  0.0624,  0.0478,  0.1157],
          [ 0.1713, -0.0242,  0.0906,  ...,  0.0377, -0.0134,  0.0919],
          [ 0.1244, -0.0675,  0.1229,  ...,  0.0393,  0.0030,  0.0779],
          ...,
          [ 0.1832, -0.0359,  0.1621,  ...,  0.0104, -0.0540,  0.1037],
          [ 0.1441, -0.0357,  0.1385,  ...,  0.0318, -0.0034,  0.1368],
          [ 0.1330, -0.0050,  0.1480,  ...,  0.0344, -0.0021,  0.0700]],
 
         [[ 0.3329, -0.1125,  0.1560,  ..., -0.0185, -0.0519,  0.2437],
          [ 0.1985, -0.0326,  0.1246,  ..., -0.0531, -0.0262,  0.2078],
          [ 0.1980, -0.0452,  0.0998,  ..., -0.0817, -0.0298,  0.1881],
          ...,
          [ 0.1929, -0.0445,  0.0991,  ..., -0.0953, -0.0030,  0.1984],
          [ 0.2284, -0.0544,  0.0959,  ..., -0.0850, -0.0146,  0.2094],
          [ 0.1768, -0.0626,  0.1009,  ..., -0.1069,  0.0117,  0.1584]]],
        grad_fn=<ViewBackward0>),
 None,
 None)

In [31]:
torch.ones(2, 2).tril(diagonal=0)

tensor([[1., 1.],
        [1., 1.]])

In [ ]:

import math
from torch import nn

class LlamaAttention(nn.Module):
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config: LlamaConfig):
        super().__init__()
        self.config = config
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.hidden_size // self.num_heads
        self.max_position_embeddings = config.max_position_embeddings

        self.q_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=False)
        self.k_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=False)
        self.v_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=False)
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=False)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        output_attentions: bool = False,
        use_cache: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        bsz, q_len, _ = hidden_states.size()

        # 获得qkv向量
        query_states = self.q_proj(hidden_states).view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
        key_states = self.k_proj(hidden_states).view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
        value_states = self.v_proj(hidden_states).view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)

        # 拼接kvcache
        if past_key_value is not None:
            key_states = torch.cat([past_key_value[0], key_states], dim=2)
            value_states = torch.cat([past_key_value[1], value_states], dim=2)

        past_key_value = (key_states, value_states) if use_cache else None

        # 计算attention权重
        attn_weights = torch.matmul(query_states, key_states.transpose(2, 3)) / math.sqrt(self.head_dim)

        # 加入mask矩阵，decoder-only为下三角
        if attention_mask is not None:
            attn_weights = attn_weights + attention_mask
            dtype_min = torch.tensor(
                torch.finfo(attn_weights.dtype).min, device=attn_weights.device, dtype=attn_weights.dtype
            )
            attn_weights = torch.max(attn_weights, dtype_min)

        # 计算softmax，这里需要从fp16升为fp32
        # upcast attention to fp32
        attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)
        attn_output = torch.matmul(attn_weights, value_states)

        attn_output = attn_output.transpose(1, 2)
        attn_output = attn_output.reshape(bsz, q_len, self.hidden_size)

        attn_output = self.o_proj(attn_output)

        if not output_attentions:
            attn_weights = None

        return attn_output, attn_weights, past_key_value

In [5]:
from torch import finfo

print(finfo(torch.float32).max)
print(finfo(torch.float32).min)
print(finfo(torch.float32).eps)
print(finfo(torch.float32).tiny)
print(finfo(torch.float32).smallest_normal)
print(finfo(torch.float32).resolution)

3.4028234663852886e+38
-3.4028234663852886e+38
1.1920928955078125e-07
1.1754943508222875e-38
1.1754943508222875e-38
1e-06
